# `*` Arguments — Advanced Practice

These problems deepen your understanding of starred positional arguments (`*args`) and starred **unpacking** in calls and assignments.

## Best practices
- Use *keyword-only* parameters after `*args` when options are required.
- Validate inputs early; raise clear exceptions (`ValueError`, `TypeError`).
- Use type hints and concise docstrings.
- Keep functions pure.
- Small, composable helpers help readability.


## Problem 1 — `safe_sum` with mixed iterables

`safe_sum(*values)` accepts any number of positional arguments where each argument is either:
- a number (int/float), or
- an **iterable of numbers** (list/tuple/set), *one level deep only*.

Strings and dicts are invalid. Empty call returns `0`.

**Examples**
- `safe_sum(1, 2, [10, 20], (0.5,)) -> 33.5`
- `safe_sum({1, 2}, 3) -> 6`


In [1]:
from typing import Iterable, Union

Number = Union[int, float]

def safe_sum(*values: object) -> Number:
    """Sum numbers and one-level iterables of numbers.
    
    Raises:
        TypeError: if an argument is unsupported, or an iterable contains a non-number.
    """
    total: float = 0.0
    for v in values:
        if isinstance(v, (int, float)):
            total += float(v)
        elif isinstance(v, (str, bytes)) or isinstance(v, dict):
            raise TypeError("Unsupported type for safe_sum: str/bytes/dict")
        else:
            try:
                iterator = iter(v)  # type: ignore[arg-type]
            except TypeError:
                raise TypeError(f"Unsupported argument type: {type(v).__name__}") from None
            for item in iterator:
                if not isinstance(item, (int, float)):
                    raise TypeError("Iterable contains non-number")
                total += float(item)
    return int(total) if total.is_integer() else total


In [2]:
# Tests for Problem 1 (right after definition)
assert safe_sum() == 0
assert safe_sum(1, 2.5, [10, 20], (0.5,)) == 34.0
assert safe_sum({1,2}, 3) in (6, 6.0)
try:
    safe_sum("hi")
    raise AssertionError("Expected TypeError for str input")
except TypeError:
    pass
try:
    safe_sum([1, "x"])  # non-number inside iterable
    raise AssertionError("Expected TypeError for bad element")
except TypeError:
    pass


## Problem 2 — Keyword-only parameter after `*args`

`centered_average(*values, *, ignore_outliers=False)`:
- Default: mean of all numbers.
- If `ignore_outliers=True`, drop a single min and max (requires ≥3 values) else `ValueError`.
- No values → `ValueError`. Non-numbers → `TypeError`.


In [3]:
from statistics import mean
from typing import Sequence

def centered_average(*values: float, ignore_outliers: bool = False) -> float:
    """Compute a mean; optionally remove one min and one max as outliers."""
    if not values:
        raise ValueError("No values provided")
    for v in values:
        if not isinstance(v, (int, float)):
            raise TypeError("All values must be numbers")
    data = list(map(float, values))
    if ignore_outliers:
        if len(data) < 3:
            raise ValueError("Need at least 3 values to drop outliers")
        data.remove(min(data))
        data.remove(max(data))
    return mean(data)


In [4]:
# Tests for Problem 2
try:
    centered_average()
    raise AssertionError("Expected ValueError on empty")
except ValueError:
    pass
assert abs(centered_average(1, 2, 3) - 2.0) < 1e-9
try:
    centered_average(1, 2, ignore_outliers=True)
    raise AssertionError("Expected ValueError with <3 values when ignoring outliers")
except ValueError:
    pass
assert abs(centered_average(1, 100, 4, 5, ignore_outliers=True) - ((4 + 5)/2)) < 1e-9


## Problem 3 — N-ary dot product

`ndot(*vectors)` computes the generalized dot product of **2 or more** equal-length vectors: `Σ_i Π_j v_j[i]`.

Examples:
- `ndot([1,2,3], [4,5,6]) -> 32`
- `ndot([1,2], [3,4], [5,6]) -> 63` (since `1*3*5 + 2*4*6 = 15 + 48 = 63`)

Require at least 2 vectors; same length; numeric elements.


In [5]:
from typing import Iterable, Sequence

def ndot(*vectors: Sequence[float]) -> float:
    """N-ary dot product across 2+ equal-length numeric vectors."""
    if len(vectors) < 2:
        raise ValueError("Provide at least two vectors")
    lengths = {len(v) for v in vectors}
    if len(lengths) != 1:
        raise ValueError("All vectors must have the same length")
    total = 0.0
    for cols in zip(*vectors):
        prod = 1.0
        for x in cols:
            if not isinstance(x, (int, float)):
                raise TypeError("Vector entries must be numeric")
            prod *= float(x)
        total += prod
    return total


In [6]:
# Tests for Problem 3
try:
    ndot([1,2,3])
    raise AssertionError("Expected ValueError with <2 vectors")
except ValueError:
    pass
assert abs(ndot([1,2,3], [4,5,6]) - 32.0) < 1e-9
assert abs(ndot([1,2], [3,4], [5,6]) - 63.0) < 1e-9
try:
    ndot([1,2], [3])
    raise AssertionError("Expected ValueError for length mismatch")
except ValueError:
    pass


## Problem 4 — `chunked` that consumes `*args`

`chunked(*values, size)` where `size` is a **keyword-only** positive integer after `*values`.

- Return a list of tuples, each of length `size` except possibly the last.
- No values → return `[]`.
- `size < 1` → raise `ValueError`.

Example: `chunked(1,2,3,4,5, size=2) -> [(1,2),(3,4),(5,)]`


In [7]:
from typing import List, Tuple

def chunked(*values: object, size: int) -> List[Tuple[object, ...]]:
    """Group *values into fixed-size chunks. Size is keyword-only."""
    if size < 1:
        raise ValueError("size must be >= 1")
    result: List[Tuple[object, ...]] = []
    current: List[object] = []
    for v in values:
        current.append(v)
        if len(current) == size:
            result.append(tuple(current))
            current.clear()
    if current:
        result.append(tuple(current))
    return result


In [8]:
# Tests for Problem 4
assert chunked(size=2) == []
assert chunked(1,2,3,4,5, size=2) == [(1,2),(3,4),(5,)]
try:
    chunked(1, size=0)
    raise AssertionError("Expected ValueError for size<1")
except ValueError:
    pass


## Problem 5 — Forwarding with a logger decorator (focus on `*args`)

Create a decorator `log_calls(fn)` that returns a wrapper accepting arbitrary positional arguments and **forwards them** to `fn` using `*args` (ignore `**kwargs` by design for this practice).

Log format: `CALL <name>(<args_repr>) -> <result_repr>`

Preserve `__name__` and `__doc__` using `functools.wraps`.


In [9]:
import functools
from typing import Callable, Any

def log_calls(fn: Callable[..., Any]) -> Callable[..., Any]:
    """Decorator that logs positional calls only (for practice)."""
    @functools.wraps(fn)
    def wrapper(*args):  # focus on *args only
        result = fn(*args)
        print(f"CALL {fn.__name__}({', '.join(map(repr, args))}) -> {result!r}")
        return result
    return wrapper

@log_calls
def add3(a, b, c):
    return a + b + c

@log_calls
def join3(a, b, c):
    return f"{a}-{b}-{c}"


In [10]:
# Tests for Problem 5
out1 = add3(1,2,3)
out2 = join3('x','y','z')
assert out1 == 6 and out2 == 'x-y-z'
assert add3.__name__ == 'add3' and join3.__name__ == 'join3'


CALL add3(1, 2, 3) -> 6
CALL join3('x', 'y', 'z') -> 'x-y-z'


## Problem 6 — `call_all(*funcs, arg)`

`call_all(*funcs, arg)` accepts a variadic list of single-argument functions and a keyword-only `arg` placed after `*funcs`.

Return a list of results, same order as `funcs`. Validate that each item is callable and accepts one positional argument.


In [11]:
from typing import Callable, List, Any

def call_all(*funcs: Callable[[Any], Any], arg: Any) -> List[Any]:
    """Call each function in *funcs with the same argument and return results."""
    results: List[Any] = []
    for fn in funcs:
        if not callable(fn):
            raise TypeError("All items must be callable")
        try:
            results.append(fn(arg))
        except TypeError as e:
            raise TypeError(f"Function {getattr(fn, '__name__', fn)!r} is not unary") from e
    return results


In [12]:
# Tests for Problem 6
assert call_all(str.strip, str.upper, arg="  hi  ") == ["hi", "  HI  "]
try:
    call_all(42, arg=10)
    raise AssertionError("Expected TypeError for non-callable")
except TypeError:
    pass


## Problem 7 — Starred assignment with validation

`extract_ends(*values)` returns `(first, middle, last)` where:
- `first` is the first arg, `last` is the last arg, `middle` is a tuple of all in-between args (possibly empty).
- Require at least **2** arguments; otherwise `ValueError`.

Implement using `first, *middle, last = values`.


In [13]:
from typing import Tuple

def extract_ends(*values: object) -> Tuple[object, Tuple[object, ...], object]:
    """Return (first, middle_tuple, last) from *values with validation."""
    if len(values) < 2:
        raise ValueError("Need at least two values")
    first, *middle, last = values
    return first, tuple(middle), last


In [14]:
# Tests for Problem 7
assert extract_ends(1,2) == (1, (), 2)
assert extract_ends(10,20,30,40) == (10, (20,30), 40)
try:
    extract_ends(1)
    raise AssertionError("Expected ValueError for <2 args")
except ValueError:
    pass


## Problem 8 — Variadic `interleave(*iterables, *, pad=None)`

`interleave(*iterables, *, pad=None)` yields items by taking one from each iterable in turn, continuing until **all** are exhausted. When an iterable runs out, use `pad` in its place for that position.

Return a `list`.

Examples:
- `interleave([1,2,3], "ab", pad=None) -> [1, 'a', 2, 'b', 3, None]`
- `interleave([1], [2,3,4], pad=0) -> [1, 2, 0, 3, 0, 4]`

If no iterables are provided, return `[]`.


In [15]:
from typing import Iterator, List, Any

def interleave(*iterables, pad=None) -> List[Any]:
    """Round-robin items from *iterables* until all are exhausted, padding missing with `pad`."""
    if not iterables:
        return []
    iters: List[Iterator] = [iter(it) for it in iterables]
    out: List[Any] = []
    active = len(iters)
    exhausted = [False] * len(iters)
    while active > 0:
        for i, it in enumerate(iters):
            if exhausted[i]:
                out.append(pad)
                continue
            try:
                out.append(next(it))
            except StopIteration:
                exhausted[i] = True
                active -= 1
                out.append(pad)
    # Remove trailing pads introduced in the final sweep
    while out and out[-1] == pad and all(exhausted):
        out.pop()
    return out


In [16]:
# Tests for Problem 8
assert interleave([1,2,3], "ab") == [1,'a',2,'b',3]
assert interleave([1], [2,3,4], pad=0) == [1,2,0,3,0,4]
assert interleave() == []


---
### All-in-one sanity check (optional)
Run once after solving to ensure everything still passes.

In [17]:
# Quick re-checks
assert safe_sum(1, [2,3]) == 6
assert abs(centered_average(2, 2, 8, 10, ignore_outliers=True) - 5.0) < 1e-9
assert abs(ndot([1,2], [3,4]) - 11.0) < 1e-9
assert abs(ndot([1,2], [3,4], [5,6]) - 63.0) < 1e-9
assert chunked(1,2,3, size=2) == [(1,2),(3,)]
assert add3(1,1,1) == 3 and join3('a','b','c') == 'a-b-c'
assert call_all(lambda x: x+1, lambda x: x*2, arg=5) == [6, 10]
assert extract_ends('a','b','c') == ('a', ('b',), 'c')
assert interleave('ab', 'xyz', pad=None) == ['a','x','b','y',None,'z']
print('All checks passed ✔️')


CALL add3(1, 1, 1) -> 3
CALL join3('a', 'b', 'c') -> 'a-b-c'
All checks passed ✔️
